## Step 1: Load, Merge, and Clean the Datasets
In this notebook, we will perform the loading of the datasets, merging both datasets, and cleaning the merged datasets.

### 1: Load the Datasets
- **AQI Dataset**: Contains daily air quality measurements across various stations in Jakarta 2010 to 2023.
- **Weather Dataset**: Contains daily weather information in jakarta for the year 2016 to 2021

We will later filter, clean, and merge these datasets for analysis.

In [1]:
import pandas as pd

# Load AQI dataset
aqi = pd.read_csv('../data/ispu_dki_all.csv')
print("AQI dataset shape:", aqi.shape)
display(aqi.head())

# Load Weather dataset
weather = pd.read_csv('../data/Jakarta_Weather_2016_to_2021.csv')
print("Weather dataset shape:", weather.shape)
display(weather.head())


AQI dataset shape: (4626, 11)


,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,2010-01-01,DKI1 (Bunderan HI),60.0,NaN,4.0,73.0,27.0,14.0,73.0,CO,SEDANG
1,2010-01-02,DKI1 (Bunderan HI),32.0,NaN,2.0,16.0,33.0,9.0,33.0,O3,BAIK
2,2010-01-03,DKI1 (Bunderan HI),27.0,NaN,2.0,19.0,20.0,9.0,27.0,PM10,BAIK
3,2010-01-04,DKI1 (Bunderan HI),22.0,NaN,2.0,16.0,15.0,6.0,22.0,PM10,BAIK
4,2010-01-05,DKI1 (Bunderan HI),25.0,NaN,2.0,17.0,15.0,8.0,25.0,PM10,BAIK


Weather dataset shape: (2192, 33)


,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"Jakarta, Indonesia",2016-01-01,90.3,75.2,82.7,101.9,75.2,88.4,76.0,81.2,...,19.6,8,NaN,2016-01-01T05:41:40,2016-01-01T18:10:03,0.72,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"96737099999,96741099999,WIII,96739099999,WIHH,..."
1,"Jakarta, Indonesia",2016-01-02,89.1,76.8,80.9,101.8,76.8,84.9,76.0,85.6,...,15.8,7,NaN,2016-01-02T05:42:11,2016-01-02T18:10:29,0.75,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"96737099999,96741099999,WIII,96739099999,WIHH,..."
2,"Jakarta, Indonesia",2016-01-03,90.9,77.0,83.1,101.2,77.0,88.8,75.5,79.1,...,15.1,7,NaN,2016-01-03T05:42:41,2016-01-03T18:10:54,0.78,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"96737099999,96741099999,WIII,WIHH,96739099999,..."
3,"Jakarta, Indonesia",2016-01-04,89.1,78.8,83.4,101.3,78.8,91.8,76.8,80.9,...,16.5,8,NaN,2016-01-04T05:43:12,2016-01-04T18:11:18,0.81,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"96741099999,WIII,96739099999,WIHH,96749099999,..."
4,"Jakarta, Indonesia",2016-01-05,90.3,77.3,83.4,103.6,77.3,90.9,76.5,80.6,...,18.7,7,NaN,2016-01-05T05:43:43,2016-01-05T18:11:42,0.84,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"96741099999,WIII,96739099999,WIHH,96749099999,..."


### 2: Filter the AQI for year 2016-2021

The AQI datasets contains the data recorded at various stations across Jakarta from 2010 to 2023.
To match the weather data (which is for 2020 only), we perform the following steps:

- Convert the `tanggal` colum to datetime format
- Filter the AQI dataset to only include rows from **2016-2021**
- Keep only relevant pollutant column for analysis

In [2]:
# Convert 'tanggal' to datetime
aqi['tanggal'] = pd.to_datetime(aqi['tanggal'])

# Filter between Jan 1, 2016 and Dec 31, 2021
mask = (aqi['tanggal'] >= '2016-01-01') & (aqi['tanggal'] <= '2021-12-31')
aqi_filtered = aqi[mask]

# Keep necessary columns including 'max'
aqi_filtered = aqi_filtered[['tanggal', 'pm10', 'pm25', 'so2', 'co', 'o3', 'no2', 'max']]

# Rename 'tanggal' to 'date' for consistency
aqi_filtered.rename(columns={'tanggal': 'date'}, inplace=True)

# Preview
aqi_filtered.head()


,date,pm10,pm25,so2,co,o3,no2,max
2188,2016-01-01,72.0,NaN,32.0,53.0,51.0,11.0,72.0
2189,2016-01-02,52.0,NaN,32.0,29.0,50.0,12.0,52.0
2190,2016-01-03,44.0,NaN,33.0,33.0,50.0,18.0,50.0
2191,2016-01-04,61.0,NaN,33.0,36.0,103.0,24.0,103.0
2192,2016-01-05,99.0,NaN,33.0,54.0,67.0,22.0,99.0


### 3: Prepare and clean the Weather Data for Merging

Before merging, we clean and prepare the weather dataset by:

- Keeping only relevant columns for air quality prediction, such as temperature, humidity, wind, and solar features
- Dropping unnecessary columns like sunrise, moonphase, station info, and descriptions
- Converting the `datetime` column into datetime format
- Renaming `datetime` to `date` for consistency with the AQI dataset

In [3]:
# Select only relevant columns for analysis
weather = weather[['datetime', 'tempmax', 'tempmin', 'temp', 'feelslike', 'dew', 'humidity', 'precip', 'precipcover', 'windspeed', 'winddir', 'cloudcover', 'uvindex', 'solarenergy']].copy()

# Ensure 'date' column is in datetime format
weather['datetime'] = pd.to_datetime(weather['datetime'])

# Rename 'datetime' to 'date' for consistency with AQI dataset
weather.rename(columns={'datetime': 'date'}, inplace=True)

# Preview the cleaned weather data
weather.head()


,date,tempmax,tempmin,temp,feelslike,dew,humidity,precip,precipcover,windspeed,winddir,cloudcover,uvindex,solarenergy
0,2016-01-01,90.3,75.2,82.7,88.4,76.0,81.2,0.998,16.67,14.5,312.2,58.2,8,19.6
1,2016-01-02,89.1,76.8,80.9,84.9,76.0,85.6,0.585,16.67,17.7,306.2,68.0,7,15.8
2,2016-01-03,90.9,77.0,83.1,88.8,75.5,79.1,0.000,0.00,11.1,292.7,56.7,7,15.1
3,2016-01-04,89.1,78.8,83.4,91.8,76.8,80.9,0.000,0.00,7.8,306.9,61.8,8,16.5
4,2016-01-05,90.3,77.3,83.4,90.9,76.5,80.6,0.457,8.33,11.4,10.7,59.0,7,18.7


### 4: Merge AQI and Weather Data

We perform an **inner join** on "date".

This combines AQI and weather data into a single dataset, with one row per day.

We will also convert the temperature columns from Fahrenheit to Celsius

The result will be used for exploratory analysis and modelling.

In [4]:
# Merge the datasets on 'date'
merged_data = pd.merge(aqi_filtered, weather, on='date', how='inner')

# Convert the temperature columns from Fahrenheit to Celsius
merged_data['tempmax'] = (merged_data['tempmax'] - 32) * 5.0/9.0
merged_data['tempmin'] = (merged_data['tempmin'] - 32) * 5.0/9.0
merged_data['temp'] = (merged_data['temp'] - 32) * 5.0/9.0
merged_data['feelslike'] = (merged_data['feelslike'] - 32) * 5.0/9.0

# Round the temperature columns to 2 decimal places
merged_data = merged_data.round({'tempmax': 2, 'tempmin': 2, 'temp': 2, 'feelslike': 2})

# Check for missing values
missing_values = merged_data.isnull().sum()
print("Missing values in merged data:\n", missing_values[missing_values > 0])

Missing values in merged data:
 pm25    1714
so2       13
co         1
dtype: int64


In [5]:
# Drop the columns pm25, since there are too many missing values, we can use pm10 or max instead
merged_data.drop(columns=['pm25'], inplace=True)

# Preview result
print("Merged data shape:", merged_data.shape)
merged_data.head()

Merged data shape: (2049, 20)


,date,pm10,so2,co,o3,no2,max,tempmax,tempmin,temp,feelslike,dew,humidity,precip,precipcover,windspeed,winddir,cloudcover,uvindex,solarenergy
0,2016-01-01,72.0,32.0,53.0,51.0,11.0,72.0,32.39,24.00,28.17,31.33,76.0,81.2,0.998,16.67,14.5,312.2,58.2,8,19.6
1,2016-01-02,52.0,32.0,29.0,50.0,12.0,52.0,31.72,24.89,27.17,29.39,76.0,85.6,0.585,16.67,17.7,306.2,68.0,7,15.8
2,2016-01-03,44.0,33.0,33.0,50.0,18.0,50.0,32.72,25.00,28.39,31.56,75.5,79.1,0.000,0.00,11.1,292.7,56.7,7,15.1
3,2016-01-04,61.0,33.0,36.0,103.0,24.0,103.0,31.72,26.00,28.56,33.22,76.8,80.9,0.000,0.00,7.8,306.9,61.8,8,16.5
4,2016-01-05,99.0,33.0,54.0,67.0,22.0,99.0,32.39,25.17,28.56,32.72,76.5,80.6,0.457,8.33,11.4,10.7,59.0,7,18.7


### 5: Save the new cleaned CSV data

In [6]:
# save the cleaned data to a new CSV file
merged_data.to_csv('../data/cleaned_aqi_weather.csv', index=False)